# EdmondKarp Algorithm
[Murwan Siddig](mailto:msiddig@clemson.edu)

In [1]:
function bfs(s,graph)
    ###############
    # Inputs
    # s: index of the node where you start the graph search
    # graph: a representation of the graph using the adjacency list
    ###############
    # Outputs
    # visitedNodeSet: a set of nodes that can be visited from s
    # pred: the precedence of each node as a result of BFS
    ###############
    n = length(graph);
    visited = fill(0,n);
    pred = fill(-1,n);
    OPEN = Any[];
    push!(OPEN, s);
    visited[s] = 1;
    while (length(OPEN) > 0)
        t = OPEN[1];
        for i in graph[t]
            if visited[i] == 0
                visited[i] = 1;
                pred[i] = t;
                push!(OPEN, i);
            end
        end
        filter!(e->e!=t, OPEN);
    end
    visitedNodeSet = Any[];
    for i=1:n
        if visited[i] == 1
            push!(visitedNodeSet, i);
        end
    end
    return pred, visitedNodeSet
end

bfs (generic function with 1 method)

In [2]:
function constructResidualGraph(graph, flow, capacity)
    ###############
    # Inputs
    # flow: current flow value for each arc in the graph, having the same structure as the adjacency list for graph.
    # graph: a representation of the original graph using the adjacency list
    # capacity: gives the capacity of each arc 
    ###############
    # Outputs
    # residualGraph: a representation of the residual graph using the adjacency list
    # label: a 0/1 label for each arc in the residualGraph, having the same structure as the adjacency list for residualGraph. This is used to denote if an arc is forward/backward in the original graph
    ###############
    residualGraph = Any[];
    label = Any[];
    for i = 1:length(graph)
        tempResidual = Any[];
        tempLabel = Any[];
        for j = 1:length(graph[i])
            if flow[i][j] < capacity[i][j]
                # forward arc
                push!(tempResidual, graph[i][j]);
                push!(tempLabel, 0);
            end
        end
        push!(residualGraph, tempResidual);
        push!(label, tempLabel);
    end
    for i = 1:length(graph)
        for j = 1:length(graph[i])
            if flow[i][j] > 0
                # backward arc
                push!(residualGraph[graph[i][j]], i);
                push!(label[graph[i][j]], 1);
            end
        end
    end
    return residualGraph, label
end

constructResidualGraph (generic function with 1 method)

In [3]:
function EdmondKarp(graph,capacity)
    ###############
    # Inputs
    # capacity: capacity of each arc in the graph, having the same structure as the adjacency list for graph.
    # graph: a representation of the original graph using the adjacency list
    ###############
    # Outputs
    # maxflow: maximum flow value
    ###############
    # First, initialize flow to be 0-flow
    flow = Any[];
    for i in graph
        subflow = fill(0,length(graph[i]));
        push!(flow, subflow);
    end
    residualGraph, label = constructResidualGraph(graph, flow, capacity);
    s = 1;
    n = length(graph);
    pred, visitedNodeSet = bfs(s,residualGraph);
    while (n in visitedNodeSet)
        # Whenever n can be found from s in BFS, meaning that we have found an augmenting path
        path = Any[];
        t = n;
        while pred[t] != -1
            segment = fill(0,2);
            segment[1] = pred[t];
            segment[2] = t;
            push!(path, segment);
            t = pred[t];
        end
        # Now go through the path, find the bottleneck
        minCap = 1e8;
        for seg in path
            # First check if it is a forward or a backward arc
            temparr = residualGraph[seg[1]];
            #ind = find(temparr -> temparr == seg[2],temparr);
            ind = findall(x -> x == seg[2],temparr)[1];
            #println("temparr = ", temparr)
            #println("seg[2] = ", seg[2])
            #ind = findfirst(temparr, seg[2]);
            cap = 0;
            if label[seg[1]][ind] == 1
                # backward
                temparr2 = graph[seg[2]];
                #ind2 = find(temparr2 -> temparr2 == seg[1], temparr2);
                ind2 = findall(x -> x == seg[1],temparr2)[1];
                #ind2 = findfirst(temparr2, seg[1]);
                cap = flow[seg[2]][ind2];
            else
                # forward
                temparr2 = graph[seg[1]];
                #ind2 = find(temparr2 -> temparr2 == seg[2], temparr2);
                ind2 = findall(x -> x == seg[2],temparr2)[1];
                #ind2 = findfirst(temparr2, seg[2]);
                cap = capacity[seg[1]][ind2] - flow[seg[1]][ind2];
            end
            if cap < minCap
                minCap = cap;
            end
        end
        # We will augment the flow now, push more minCap
        for seg in path
            # First check if it is a forward or a backward arc
            temparr = residualGraph[seg[1]];
            #ind = find(temparr -> temparr == seg[2],temparr);
            ind = findall(x -> x == seg[2],temparr)[1];
            #ind = findfirst(temparr, seg[2]);
            if label[seg[1]][ind] == 1
                # backward
                temparr2 = graph[seg[2]];
                #ind2 = find(temparr2 -> temparr2 == seg[1], temparr2);
                ind2 = findall(x -> x == seg[1],temparr2)[1];
                #ind2 = findfirst(temparr2, seg[1]);
                flow[seg[2]][ind2] -= minCap;
            else
                # forward
                temparr2 = graph[seg[1]];
                #ind2 = find(temparr2 -> temparr2 == seg[2], temparr2);
                ind2 = findall(x -> x == seg[2],temparr2)[1];
                #ind2 = findfirst(temparr2, seg[2]);
                flow[seg[1]][ind2] += minCap;
            end
        end
        # We will update the residualGraph now
        residualGraph, label = constructResidualGraph(graph, flow,capacity);
        pred, visitedNodeSet = bfs(s,residualGraph);
    end
    # After the loop, now calculate maxflow from flow
    maxflow = 0;
    for i in flow[s]
        maxflow += i;
    end
    return flow, maxflow
end

EdmondKarp (generic function with 1 method)

In [4]:
graph = [[2,3],[4],[2,4,5],[5,6],[6],[]];
capacity = [[6,4],[5],[3,4,3],[2,4],[6],[]];

In [5]:
flow, maxflow = EdmondKarp(graph,capacity);
println("maxflow = ", maxflow)

maxflow = 9
